# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise
    # не gain по gini, а мера с тем же argmin:
    # отброшены константные слагаемое и множитель
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (-np.sum(l_c ** 2, axis = 1).reshape(-1,1)/l_s - np.sum(r_c ** 2, axis = 1).reshape(-1,1)/r_s)

    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:(max(int(np.sqrt(n_feature)), 1))] # Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:(max(int(np.log2(n_feature)), 1))] # Ваш код

    def __get_feature_ids_N(self, n_feature):
        return feature_ids[:(max(int(n_feature), 1))] # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода? 
        # Блок кода упорядочивает x (значения признака для объектов) 
        # по возрастанию и y (классы объектов) в соответствующем порядке, 
        # запоминает количество уникальных значений в y как количество классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Блок кода находит соседние по значению признака объекты разных классов
        # (точки последовательности классов упорядоченных объектов sorted_y,
        # в которых произошла смена значения);
        # но лишь те, для которых потенциальное разделение узла удовлетворяет
        # ограничению по количеству объектов (min_samples_split).
        # Если объектов слишком мало для разделения или класс совпадает,
        # пороговым значением признака назначаем +inf
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Блок кода получает таблицу class_increments, в которой строки
        # соответствуют последовательным блокам из одинаковых классов в sorted_y  
        # (кроме первой: в ней учтены классы первых min_samples_split объектов,
        # которые не могут быть отсечены от остальных, вместе с первым из
        # идущих дальше блоком объектов одного класса).
        # Номер столбца - ярлык класса, номер строки - номер блока объектов.
        # (Таблица получена из бинарного кодирования классов блоков one_hot_code
        # того же строения навешиванием счетчика объектов в блоках eq_el_count)
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Подсчитывает количество объектов каждого класса и общее количество 
        # объектов в потенциальных потомках (для разбиений по границе блока 
        # из одинаковых классов) 
        # Количество объектов по классам  задается таблицами l_class_count, 
        # r_class_count той же структуры, что и  class_increments (в строке
        # частичные суммы по рядам)
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_coount
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Вызывает подсчёт gain для потенциальных разбиений и находит
        # номер блока объектов, разбиение по которому даёт лучшее
        # уменьшение неопределенности (название конкретного критерия
        # в self.G_function)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # Находит элемент, по которому производится рабиение 
        # (границу блока объектов, соответсвующую лучшему уменьшению 
        # неопределенномти), возвращает gain для этого разбиения,
        # пороговое значение признака как среднее для найденного 
        # элемента и его соседа
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.00544095039368
3.1287920475


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

1.0
1.0
0.999916853746
1.0
1.0


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.893572794546
0.894487403342
0.891327845681
0.891161553172
0.890741279674


In [10]:
print x, y
sorted_x, sorted_y = x[x.argsort()], y[x.argsort()]
print x, y
class_number = np.unique(y).shape[0]
print np.unique(x)
print np.unique(x).shape
print class_number


[[  0.76612661  45.           2.         ...,   6.           0.           2.        ]
 [  0.95715102  40.           0.         ...,   0.           0.           1.        ]
 [  0.65818014  38.           1.         ...,   0.           0.           0.        ]
 ..., 
 [  0.29974515  44.           0.         ...,   1.           0.           2.        ]
 [  0.          30.           0.         ...,   0.           0.           0.        ]
 [  0.85028295  64.           0.         ...,   2.           0.           0.        ]] [1 0 0 ..., 0 0 0]
[[  0.76612661  45.           2.         ...,   6.           0.           2.        ]
 [  0.95715102  40.           0.         ...,   0.           0.           1.        ]
 [  0.65818014  38.           1.         ...,   0.           0.           0.        ]
 ..., 
 [  0.29974515  44.           0.         ...,   1.           0.           2.        ]
 [  0.          30.           0.         ...,   0.           0.           0.        ]
 [  0.85028295  64. 

In [11]:
x.shape

(120269, 10)

In [12]:
x[x.argsort()].shape

(120269, 10, 10)

In [13]:
sample = np.asarray([[1,2,3],[3,2,1], [2,3,1], [1,3,2]])
sample

array([[1, 2, 3],
       [3, 2, 1],
       [2, 3, 1],
       [1, 3, 2]])

In [14]:
sample.argsort()

array([[0, 1, 2],
       [2, 1, 0],
       [2, 0, 1],
       [0, 2, 1]])

In [15]:
sample[np.argsort(sample)]

array([[[1, 2, 3],
        [3, 2, 1],
        [2, 3, 1]],

       [[2, 3, 1],
        [3, 2, 1],
        [1, 2, 3]],

       [[2, 3, 1],
        [1, 2, 3],
        [3, 2, 1]],

       [[1, 2, 3],
        [2, 3, 1],
        [3, 2, 1]]])

In [16]:
sample = np.array([1, 2, 3, 5, 6, 4])

In [17]:
sample[sample.argsort()]

array([1, 2, 3, 4, 5, 6])

In [58]:
sample = np.array([0,1,1,0,0,2,0,0,0,0,0,1,1,1,0,0,1,1,1])
sample

array([0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1])

In [59]:
splitted = sample[2:-2]
splitted

array([1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1])

In [60]:
x1 = np.array(splitted[:-1])
x1

array([1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0])

In [61]:
x2 = np.array(splitted[1:])
x2

array([0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1])

In [62]:
np.where(x1 != x2)

(array([ 0,  2,  3,  8, 11, 13]),)

In [63]:
x1 != x2

array([ True, False,  True,  True, False, False, False, False,  True,
       False, False,  True, False,  True], dtype=bool)

In [64]:
r_border_ids = np.where(x1 != x2)[0] + 2 + 1
r_border_ids

array([ 3,  5,  6, 11, 14, 16])

In [65]:
np.append([2], r_border_ids[:-1])

array([ 2,  3,  5,  6, 11, 14])

In [66]:
r_border_ids - np.append([2], r_border_ids[:-1])

array([1, 2, 1, 5, 3, 2])

In [67]:
one_hot_code = np.zeros((r_border_ids.shape[0], 3))
sample[r_border_ids - 1]

array([1, 0, 2, 0, 1, 0])

In [68]:
one_hot_code[np.arange(r_border_ids.shape[0]), sample[r_border_ids - 1]] = 1

In [70]:
one_hot_code

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.]])

In [41]:
eq_el_count = r_border_ids - np.append([2], r_border_ids[:-1])

In [72]:
increments = one_hot_code * eq_el_count.reshape(-1, 1)
increments

array([[ 0.,  1.,  0.],
       [ 2.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 5.,  0.,  0.],
       [ 0.,  3.,  0.],
       [ 2.,  0.,  0.]])

In [74]:
np.bincount(sample[:2], minlength=3)

array([1, 1, 0])

In [75]:
increments[0] = increments[0] + np.bincount(sample[:2], minlength=3)
increments

array([[ 1.,  2.,  0.],
       [ 2.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 5.,  0.,  0.],
       [ 0.,  3.,  0.],
       [ 2.,  0.,  0.]])

In [77]:
l_class_count = np.cumsum(increments, axis=0)        
l_class_count

array([[  1.,   2.,   0.],
       [  3.,   2.,   0.],
       [  3.,   2.,   1.],
       [  8.,   2.,   1.],
       [  8.,   5.,   1.],
       [ 10.,   5.,   1.]])

In [87]:
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
l_sizes

array([[ 3],
       [ 5],
       [ 6],
       [11],
       [14],
       [16]])

In [84]:
l_class_count **2

array([[   1.,    4.,    0.],
       [   9.,    4.,    0.],
       [   9.,    4.,    1.],
       [  64.,    4.,    1.],
       [  64.,   25.,    1.],
       [ 100.,   25.,    1.]])

In [89]:
np.sum(l_class_count ** 2, axis = 1).reshape(-1,1)

array([[   5.],
       [  13.],
       [  14.],
       [  69.],
       [  90.],
       [ 126.]])

In [90]:
np.sum(l_class_count ** 2, axis = 1).reshape(-1,1)/l_sizes

array([[ 1.66666667],
       [ 2.6       ],
       [ 2.33333333],
       [ 6.27272727],
       [ 6.42857143],
       [ 7.875     ]])

In [92]:
r_class_count = np.bincount(sample) - l_class_count
r_class_count

array([[ 9.,  6.,  1.],
       [ 7.,  6.,  1.],
       [ 7.,  6.,  0.],
       [ 2.,  6.,  0.],
       [ 2.,  3.,  0.],
       [ 0.,  3.,  0.]])

In [93]:
r_sizes = sample.shape[0] - l_sizes
r_sizes

array([[16],
       [14],
       [13],
       [ 8],
       [ 5],
       [ 3]])

In [94]:
np.sum(r_class_count ** 2, axis = 1).reshape(-1,1)/r_sizes

array([[ 7.375     ],
       [ 6.14285714],
       [ 6.53846154],
       [ 5.        ],
       [ 2.6       ],
       [ 3.        ]])

In [98]:
gini = -np.sum(l_class_count ** 2, axis = 1).reshape(-1,1)/l_sizes - np.sum(r_class_count ** 2, axis = 1).reshape(-1,1)/r_sizes
gini

array([[ -9.04166667],
       [ -8.74285714],
       [ -8.87179487],
       [-11.27272727],
       [ -9.02857143],
       [-10.875     ]])

In [99]:
np.argmin(gini)


3

In [100]:
l_sizes[3][0]

11

In [95]:
r_class_count + l_class_count

array([[ 10.,   8.,   1.],
       [ 10.,   8.,   1.],
       [ 10.,   8.,   1.],
       [ 10.,   8.,   1.],
       [ 10.,   8.,   1.],
       [ 10.,   8.,   1.]])

In [80]:
r_border_ids.reshape(l_class_count.shape[0], 1)

array([[ 3],
       [ 5],
       [ 6],
       [11],
       [14],
       [16]])

In [83]:
sample.shape[0] - r_border_ids.reshape(l_class_count.shape[0], 1)

array([[16],
       [14],
       [13],
       [ 8],
       [ 5],
       [ 3]])

In [ ]:
r_class_count = np.bincount(y) - l_class_count
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
r_sizes = sorted_y.shape[0] - l_sizes

In [28]:
sample = [0,0,0,0,0,0,0,0,0]

In [29]:
splitted = sample[2:-2]
splitted

[0, 0, 0, 0, 0]

In [30]:
np.where(splitted[:-1] != splitted[1:])[0] + 2 + 1

array([], dtype=int64)

In [ ]:
class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)

In [31]:
splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        

NameError: name 'self' is not defined

In [ ]:
eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)
        